# Virtual Try-On Leffa Loss - Dependency Conflict Resolution

This notebook helps resolve Python dependency conflicts when running the Virtual Try-On Leffa Loss project in Google Colab.

## Problem Overview
You're getting dependency conflicts because:
1. **Environment Differences**: Colab vs Kaggle have different pre-installed packages
2. **NumPy Version Issues**: Many packages require NumPy < 2.0, but you have 2.2.6
3. **Missing Version Constraints**: The original requirements.txt lacks specific versions

## Solution Strategy
We'll systematically check, analyze, and fix the dependency conflicts step by step.

## 1. Check Current Environment

First, let's examine what packages are currently installed and their versions.

In [ ]:
# Check Python version
import sys
print(f"Python version: {sys.version}")

# Check if we're in Colab
try:
    import google.colab
    print("✅ Running in Google Colab")
    IN_COLAB = True
except ImportError:
    print("❌ Not running in Google Colab")
    IN_COLAB = False

In [ ]:
# Check versions of problematic packages
problematic_packages = [
    'numpy', 'scipy', 'scikit-learn', 'matplotlib', 'pandas', 
    'tensorflow', 'torch', 'torchvision', 'protobuf', 'pyarrow'
]

import subprocess
import pkg_resources

for package in problematic_packages:
    try:
        version = pkg_resources.get_distribution(package).version
        print(f"{package}: {version}")
    except pkg_resources.DistributionNotFound:
        print(f"{package}: Not installed")

## 2. Analyze Dependency Conflicts

Let's check for dependency conflicts and understand the dependency tree.

In [ ]:
# Install pipdeptree to analyze dependencies
!pip install pipdeptree --quiet

# Check for dependency conflicts
print("=== Checking for dependency conflicts ===")
!pip check

In [ ]:
# Show dependency tree for key packages
print("=== Dependency tree for numpy ===")
!pipdeptree -p numpy

print("\n=== Dependency tree for scipy ===")
!pipdeptree -p scipy

print("\n=== Dependency tree for tensorflow ===")
!pipdeptree -p tensorflow

## 3. Compare Package Versions

Let's compare what we have vs what's needed for the Virtual Try-On project.

In [ ]:
# Required packages for Virtual Try-On Leffa Loss with compatible versions
required_packages = {
    'numpy': '1.26.4',  # Must be < 2.0 for compatibility
    'scipy': '1.13.1',   # Must be < 1.14.0 for gensim
    'scikit-learn': '1.5.1',  # Must be >= 1.3.2 for other packages
    'matplotlib': '3.8.4',    # Must be >= 3.8.0 for plotnine
    'pandas': '2.2.2',        # Required by google-colab
    'torch': '2.1.0',         # Stable version with CUDA support
    'torchvision': '0.16.0',  # Compatible with torch 2.1.0
    'tensorflow': '2.15.0',   # Compatible with numpy < 2.0
    'protobuf': '4.25.3',     # Must be >= 4.25.1 for onnx
    'pyarrow': '15.0.2',      # Must be >= 21.0.0 for datasets
}

print("=== Required vs Current Versions ===")
for package, required_version in required_packages.items():
    try:
        current_version = pkg_resources.get_distribution(package).version
        status = "✅" if current_version == required_version else "❌"
        print(f"{package}: Required {required_version}, Current {current_version} {status}")
    except pkg_resources.DistributionNotFound:
        print(f"{package}: Required {required_version}, Current Not installed ❌")

## 4. Install Compatible Packages

Now let's install the packages with compatible versions step by step.

⚠️ **Warning**: This will restart your runtime after installation.

In [ ]:
# Step 1: Install core scientific packages with compatible versions
print("Installing core scientific packages...")
!pip install numpy==1.26.4 --force-reinstall --quiet
!pip install scipy==1.13.1 --force-reinstall --quiet
!pip install scikit-learn==1.5.1 --force-reinstall --quiet
!pip install matplotlib==3.8.4 --force-reinstall --quiet
!pip install pandas==2.2.2 --force-reinstall --quiet

print("✅ Core scientific packages installed")

In [ ]:
# Step 2: Install PyTorch ecosystem
print("Installing PyTorch ecosystem...")
!pip install torch==2.1.0 torchvision==0.16.0 torchaudio==2.1.0 --index-url https://download.pytorch.org/whl/cu118 --quiet

print("✅ PyTorch ecosystem installed")

In [ ]:
# Step 3: Install TensorFlow with compatible version
print("Installing TensorFlow...")
!pip install tensorflow==2.15.0 --quiet

print("✅ TensorFlow installed")

In [ ]:
# Step 4: Install specific packages that were causing conflicts
print("Installing packages that were causing conflicts...")
!pip install protobuf==4.25.3 --quiet
!pip install pyarrow==15.0.2 --quiet

print("✅ Conflict-causing packages fixed")

In [ ]:
# Step 5: Install the remaining Virtual Try-On requirements
print("Installing remaining Virtual Try-On requirements...")

packages_to_install = [
    "transformers==4.40.0",
    "diffusers==0.27.2", 
    "accelerate==0.30.1",
    "gradio==4.26.0",
    "opencv-python==4.9.0.80",
    "scikit-image==0.22.0",
    "pillow==10.3.0",
    "omegaconf==2.3.0",
    "einops==0.7.0",
    "fvcore==0.1.5",
    "pycocotools==2.0.7",
    "timm==0.9.16",
    "torchmetrics==1.4.0",
    "safetensors==0.4.3",
    "tokenizers==0.19.1",
    "peft==0.11.1",
    "regex==2024.5.15",
    "cloudpickle==3.0.0",
    "onnxruntime==1.18.0",
    "imageio==2.34.1",
    "tqdm==4.66.4",
    "psutil==5.9.8"
]

for package in packages_to_install:
    print(f"Installing {package}...")
    !pip install {package} --quiet

print("✅ All Virtual Try-On requirements installed!")

## 5. Restart Runtime

**IMPORTANT**: After running the above installation cells, you need to restart your runtime to ensure all packages are properly loaded.

In [ ]:
# Restart the runtime automatically
import os
os.kill(os.getpid(), 9)

## 6. Verify Installation

After the runtime restarts, run the following cells to verify everything is working correctly.

In [ ]:
# Test importing all critical packages
test_packages = [
    'numpy', 'scipy', 'sklearn', 'matplotlib', 'pandas',
    'torch', 'torchvision', 'tensorflow', 'transformers', 
    'diffusers', 'gradio', 'cv2', 'PIL'
]

print("=== Testing Package Imports ===")
for package in test_packages:
    try:
        if package == 'sklearn':
            import sklearn
        elif package == 'cv2':
            import cv2
        elif package == 'PIL':
            import PIL
        else:
            __import__(package)
        print(f"✅ {package}: Successfully imported")
    except ImportError as e:
        print(f"❌ {package}: Import failed - {e}")

In [ ]:
# Final dependency check
print("=== Final Dependency Check ===")
!pip check

print("\n=== Package Versions After Fix ===")
import pkg_resources

for package in ['numpy', 'scipy', 'scikit-learn', 'torch', 'tensorflow']:
    try:
        version = pkg_resources.get_distribution(package).version
        print(f"{package}: {version}")
    except pkg_resources.DistributionNotFound:
        print(f"{package}: Not installed")

In [ ]:
# Test running the Virtual Try-On app
print("=== Testing Virtual Try-On App Import ===")
try:
    # Clone or upload your project files first, then:
    # import sys
    # sys.path.append('/content/your-project-path')
    
    # Test basic functionality
    import numpy as np
    import torch
    from PIL import Image
    
    print("✅ Basic imports successful")
    print(f"NumPy version: {np.__version__}")
    print(f"PyTorch version: {torch.__version__}")
    print(f"CUDA available: {torch.cuda.is_available()}")
    
    # You can now run your Virtual Try-On app!
    print("\n🎉 Ready to run Virtual Try-On Leffa Loss!")
    
except Exception as e:
    print(f"❌ Error: {e}")
    print("Please check if all packages are correctly installed.")

## Summary

### Why Your Friend's Kaggle Setup Works but Yours Doesn't:

1. **Different Pre-installed Packages**: Kaggle and Google Colab have different default environments
2. **NumPy Version**: Colab has NumPy 2.2.6, but many ML packages require NumPy < 2.0
3. **Environment Conflicts**: Google Colab has strict package version requirements that conflict with newer packages

### What This Notebook Fixed:

1. ✅ Downgraded NumPy to 1.26.4 (compatible with all packages)
2. ✅ Installed scipy 1.13.1 (compatible with gensim requirements)
3. ✅ Upgraded scikit-learn to 1.5.1 (meets modern package requirements)
4. ✅ Used compatible PyTorch and TensorFlow versions
5. ✅ Fixed protobuf and pyarrow version conflicts

### Next Steps:

1. Upload your Virtual Try-On project files to Colab
2. Run the verification cells above
3. Your project should now work without dependency conflicts!

🎉 **Happy Virtual Try-On modeling!**